<a href="https://colab.research.google.com/github/seongheek/econtheory/blob/main/14%EC%A3%BC%EC%B0%A8_%EA%B0%95%EC%9D%98_%ED%8C%A8%EB%84%90%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **패널분석**
패널데이터를 사용하여 패널 분석을 연습해보자.

In [1]:
#함수 불러오기
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
#데이터 불러오기
!pip install linearmodels
from linearmodels.datasets import wage_panel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.7 MB/s eta 0:00:00


In [2]:
# wage_panel 데이터 로드 및 DataFrame으로 변환
df= wage_panel.load()
df

,nr,year,black,exper,hisp,hours,married,educ,union,lwage,expersq,occupation
0,13,1980,0,1,0,2672,0,14,0,1.197540,1,9
1,13,1981,0,2,0,2320,0,14,1,1.853060,4,9
2,13,1982,0,3,0,2940,0,14,0,1.344462,9,9
3,13,1983,0,4,0,2960,0,14,0,1.433213,16,9
4,13,1984,0,5,0,3071,0,14,0,1.568125,25,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4355,12548,1983,0,8,0,2080,1,9,0,1.591879,64,5
4356,12548,1984,0,9,0,2080,1,9,1,1.212543,81,5
4357,12548,1985,0,10,0,2080,1,9,0,1.765962,100,5
4358,12548,1986,0,11,0,2080,1,9,1,1.745894,121,5


nr이 i(개인), year이 t(시간)인 패널 형태의 데이터셋인 것으로 파악된다.

연평균 로그 임금수준을 계산해보자.

In [3]:
df.groupby("year")["lwage"].mean()

,lwage
year,
1980,1.393477
1981,1.512867
1982,1.571667
1983,1.619263
1984,1.690295
1985,1.739410
1986,1.799719
1987,1.866479


먼저 OLS로 회귀모형을 돌려보자.

In [4]:
mod = smf.ols("lwage ~ exper + union + married + hours + black + hisp + educ", data=df).fit()
mod.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.1259,0.066,1.901,0.057,-0.004,0.256
exper,0.0527,0.003,17.960,0.000,0.047,0.058
union,0.1783,0.017,10.421,0.000,0.145,0.212
married,0.1240,0.016,7.853,0.000,0.093,0.155
hours,-6.659e-05,1.33e-05,-4.989,0.000,-9.28e-05,-4.04e-05
black,-0.1412,0.024,-6.001,0.000,-0.187,-0.095
hisp,0.0167,0.021,0.805,0.421,-0.024,0.057
educ,0.1055,0.005,23.115,0.000,0.097,0.114


위의 모형에서 연도 더미를 추가해보자.

In [5]:
mod = smf.ols("lwage ~ exper + union + married + hours + black + hisp + educ + C(year)", data=df).fit()
mod.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.2750,0.078,3.509,0.000,0.121,0.429
C(year)[T.1981],0.0834,0.030,2.821,0.005,0.025,0.141
C(year)[T.1982],0.1040,0.031,3.354,0.001,0.043,0.165
C(year)[T.1983],0.1182,0.033,3.545,0.000,0.053,0.184
C(year)[T.1984],0.1539,0.036,4.239,0.000,0.083,0.225
C(year)[T.1985],0.1729,0.040,4.348,0.000,0.095,0.251
C(year)[T.1986],0.2019,0.044,4.627,0.000,0.116,0.287
C(year)[T.1987],0.2262,0.048,4.735,0.000,0.133,0.320
exper,0.0316,0.005,5.760,0.000,0.021,0.042
union,0.1809,0.017,10.579,0.000,0.147,0.214


고정효과를 사용하려면 panelOLS 함수를 불러온다. 일단 i와 t를 인덱스로 설정해준다.
(nr과 year이 변수가 아닌 인덱스 처리되므로 나중에 OLS 등을 다시 돌리고 싶을 때는 df = df.reset_index() 명령어로 인덱스화 된 것을 풀어줘야 한다.)

In [6]:
from linearmodels.panel import PanelOLS

df=df.set_index(["nr", "year"])

고정효과 모형은 변하지 않는 변수(성별, 인종 등)은 추정할 수 없다. 따라서 모형에서 빼준다.

In [7]:
mod = PanelOLS(df.lwage, df[["exper", "union", "married", "hours"]], entity_effects=True)

# 클러스터링 표준오차로 회귀 추정
result = mod.fit(cov_type='clustered', cluster_entity=True)

# 회귀계수 표 출력
result.summary.tables[1]

,Parameter,Std. Err.,T-stat,P-value,Lower CI,Upper CI
exper,0.0660,0.0034,19.418,0.0000,0.0594,0.0727
union,0.0781,0.0228,3.4300,0.0006,0.0335,0.1228
married,0.0649,0.0214,3.0355,0.0024,0.0230,0.1069
hours,-0.0001,2.146e-05,-5.4010,0.0000,-0.0002,-7.384e-05


FE에는 time effects=True까지 넣는 것이 바람직하나, 관심변수가 시간과 함께 움직이는 변수라면(exper) 추정할 수 없다. unit fixed effect 뿐만 아니라 time fixed effect를 고려하는 모델을 two-way fixed effect 모델이라고 한다. 자동으로 변수를 제거해주도록 하고 싶으면 drop_absorbed=True 옵션을 넣어준다.

In [8]:
mod = PanelOLS(df.lwage, df[["exper", "hisp", "union", "married", "hours"]], entity_effects=True, drop_absorbed=True)

# 클러스터링 표준오차로 회귀 추정
result = mod.fit(cov_type='clustered', cluster_entity=True)

# 회귀계수 표 출력
summary_table=result.summary.tables[1]


<ipython-input-8-c59aee734bc4>:4: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

hisp

  result = mod.fit(cov_type='clustered', cluster_entity=True)


경력(exper)을 추정하고 싶다면 제곱시킨 expersq를 대신 넣어주고, entity 와 time effects를 둘 다 넣어준다.

In [9]:
mod = PanelOLS(df.lwage, df[["expersq", "union", "married", "hours"]], entity_effects=True, drop_absorbed=True)

# 클러스터링 표준오차로 회귀 추정
result = mod.fit(cov_type='clustered', cluster_entity=True)

# 회귀계수 표 출력
result.summary.tables[1]

,Parameter,Std. Err.,T-stat,P-value,Lower CI,Upper CI
expersq,0.0040,0.0002,16.552,0.0000,0.0035,0.0044
union,0.0784,0.0236,3.3225,0.0009,0.0322,0.1247
married,0.1147,0.0220,5.2213,0.0000,0.0716,0.1577
hours,-8.46e-05,2.22e-05,-3.8105,0.0001,-0.0001,-4.107e-05


Random Effects를 돌려보자. RE의 장점은 시간에 따라 변하지 않는 변수를 분석할 수 있다는 것에 있다. 그렇지 않은 상황이라면 FE를 사용하는 것이 일반적이다.

In [10]:
import statsmodels.api as sm
#RE 불러오기
from linearmodels.panel import RandomEffects


# 변수 정의
y = df["lwage"]
X = sm.add_constant(df[["exper", "union", "married", "hours", "black", "hisp", "educ"]])

# 랜덤효과 모형 설정
mod = RandomEffects(y, X)

# 회귀 추정
result = mod.fit()

# 결과 출력
print(result.summary)

                        RandomEffects Estimation Summary                        
Dep. Variable:                  lwage   R-squared:                        0.1821
Estimator:              RandomEffects   R-squared (Between):              0.1786
No. Observations:                4360   R-squared (Within):               0.1826
Date:                Wed, Jun 04 2025   R-squared (Overall):              0.1804
Time:                        08:14:59   Log-likelihood                   -1619.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      138.40
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                  F(7,4352)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             138.40
                            

Hausman Test 시행하기.
동일한 모형으로, FE와 RE 중 어떤 것이 적합한지 검정해보자.

In [11]:
#Hausman test 정의하기

from scipy.stats import chi2
import numpy as np

def hausman_test(fe_res, re_res, verbose=True):
    b_FE = fe_res.params
    b_RE = re_res.params
    common = b_FE.index.intersection(b_RE.index)
    b_diff = b_FE[common] - b_RE[common]
    V_FE = fe_res.cov.loc[common, common]
    V_RE = re_res.cov.loc[common, common]
    V_diff = V_FE - V_RE
    stat = b_diff.T @ np.linalg.inv(V_diff) @ b_diff
    df_h = len(b_diff)
    pval = 1 - chi2.cdf(stat, df_h)
    if verbose:
        print(f"Hausman test statistic = {stat:.4f}")
        print(f"Degrees of freedom = {df_h}")
        print(f"P-value = {pval:.4f}")


In [12]:
from linearmodels.panel import compare

y = df["lwage"]
X = sm.add_constant(df[["expersq", "union", "married", "hours"]])

# 고정효과 추정
fe_model = PanelOLS(y, X, entity_effects=True, drop_absorbed=True)
fe_res = fe_model.fit(cov_type='unadjusted')   #Hausman 검정은 공분산 차이를 비교하므로, 조정되지 않은 표준오차를 사용한다.

# 랜덤효과 추정
re_model = RandomEffects(y, X)
re_res = re_model.fit()

# 두 모형 비교
comparison = compare({'Fixed Effects': fe_res, 'Random Effects': re_res})
print(comparison)

                    Model Comparison                    
                         Fixed Effects    Random Effects
--------------------------------------------------------
Dep. Variable                    lwage             lwage
Estimator                     PanelOLS     RandomEffects
No. Observations                  4360              4360
Cov. Est.                   Unadjusted        Unadjusted
R-squared                       0.1454            0.1205
R-Squared (Within)              0.1454            0.1432
R-Squared (Between)            -0.0844           -0.0484
R-Squared (Overall)             0.0219            0.0402
F-statistic                     162.14            149.21
P-value (F-stat)                0.0000            0.0000
=====================     ============   ===============
const                           1.5658            1.5499
                              (52.777)          (47.793)
expersq                         0.0040            0.0034
                              (

In [13]:
hausman_test(fe_res, re_res)

Hausman test statistic = 124.2773
Degrees of freedom = 5
P-value = 0.0000


유의하지 않은 차이라면, 랜덤효과를 사용한다.
예를 들어,
p < 0.05 → 고정효과(FE) 선호
p > 0.05 → 랜덤효과(RE) 사용 가능
이라는 기준을 삼아도 된다.
결과상 두 모형간 유의한 차이가 나므로, 고정효과를 사용하는 것이 낫다. 하지만 일반적으로 패널 데이터 사용시 하우스만 검정 과정을 생략하고 고정효과 모형을 채택한다.